In [1]:
import os
# os.getcwd()
os.chdir('/dev-data/zzz/supplementary/casr/')

In [ ]:
self.USER_ID = config['USER_ID_FIELD']
self.ITEM_ID = config['ITEM_ID_FIELD']
self.ITEM_SEQ = self.ITEM_ID + config['LIST_SUFFIX']
self.ITEM_SEQ_LEN = config['ITEM_LIST_LENGTH_FIELD']
self.POS_ITEM_ID = self.ITEM_ID
self.NEG_ITEM_ID = config['NEG_PREFIX'] + self.ITEM_ID
self.max_seq_length = config['MAX_ITEM_LIST_LENGTH']
self.n_items = dataset.num(self.ITEM_ID)
# load parameters info
self.n_layers = config['n_layers']
self.n_heads = config['n_heads']
self.hidden_size = config['hidden_size']  # same as embedding_size
self.inner_size = config['inner_size']  # the dimensionality in feed-forward layer
self.attribute_hidden_size = config['attribute_hidden_size']
self.hidden_dropout_prob = config['hidden_dropout_prob']
self.attn_dropout_prob = config['attn_dropout_prob']
self.hidden_act = config['hidden_act']
self.layer_norm_eps = config['layer_norm_eps']

self.selected_features = config['selected_features']
self.pooling_mode = config['pooling_mode']
self.device = config['device']
self.num_feature_field = len(config['selected_features'])

self.initializer_range = config['initializer_range']
self.loss_type = config['loss_type']
self.fusion_type = config['fusion_type']

self.lamdas = config['lamdas']
self.attribute_predictor = config['attribute_predictor']

In [2]:
import torch

tl = 10
attention_mask = torch.tril(torch.ones((tl, tl), dtype=torch.bool))
attention_mask

tensor([[ True, False, False, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])

In [11]:
import torch
import numpy as np
import pytorch_lightning as pl
from torch.utils.data import *
import sys
import os
import datetime
import pickle

from new_utils import *





def str2bool(s):
    if s not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s == 'true'
import argparse
import datetime



parser = argparse.ArgumentParser()


parser.add_argument('--batch_size', default=128, type=int) #mat1(a,b)->a
parser.add_argument('--beam_size', type=int, default=10)


parser.add_argument('--cuda', type=int, default=0)
parser.add_argument('--data_name', default=None, type=str)

parser.add_argument('--dataset',  default='None', type=str)
parser.add_argument('--datasets',  default='sub_taobao', type=str)
parser.add_argument('--device', default='cuda', type=str)
parser.add_argument('--dropout_rate', default=0.25, type=float)
parser.add_argument("--embedding_dim", type=int, default=64,
                     help="using embedding")
parser.add_argument('--hidden_units', default=64, type=int)
parser.add_argument('--hidden_size', default=64, type=int)
parser.add_argument('--incatemaxlen', type=int, default=10)


parser.add_argument('--lastk', type=int, default=5)
parser.add_argument('--lr', default=.001, type=float)
parser.add_argument('--l2_emb', default=0.005, type=float)
parser.add_argument('--maxlen', type=int, default=50)
parser.add_argument('--model', type=str, default='casr')


parser.add_argument('--num_layers', default=2, type=int)
parser.add_argument('--num_epochs', default=100, type=int)
parser.add_argument('--num_heads', default=2, type=int)
parser.add_argument('--num_blocks', default=2, type=int)


parser.add_argument('--optimizer_type', default='Adagrad', type=str)
parser.add_argument('--recall', default=[5, 10, 20, 50], type=list)



parser.add_argument("--seed", type=int, default=5,
                     help="Seed for random initialization")


parser.add_argument("-sigma", type=float, default=None,
                     help="init weight -1: range [-sigma, sigma], -2: range [0, sigma]")
parser.add_argument('--state_dict_path', default=None, type=str)

parser.add_argument('--time_bucket', type=int, default=64)
parser.add_argument('--time_span', type=int, default=64)
parser.add_argument('--topkc', default=1, type=int)


args, _ = parser.parse_known_args()

np.random.seed(args.seed)
if args.device == 'cuda':
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

def init_model(model):
    if args.sigma is not None:
        for p in model.parameters():
            if args.sigma != -1 and args.sigma != -2:
                sigma = args.sigma
                p.data.uniform_(-sigma, sigma)
            elif len(list(p.size())) > 1:
                sigma = np.sqrt(6.0 / (p.size(0) + p.size(1)))
                if args.sigma == -1:
                    p.data.uniform_(-sigma, sigma)
                else:
                    p.data.uniform_(0, sigma)

def count_parameters(model):
    parameter_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("parameter_num", parameter_num) 


batch_size = args.batch_size
lr = args.lr






In [3]:



st = datetime.datetime.now()

if args.datasets =='sub_taobao':
    args.dataset = './data/subtb_10_50.csv'
    User = dataall_partition_last(args.dataset)
else:
    print('wrong datasets')

directory = './dataset/'+args.datasets+'/'

if not os.path.exists(directory):
    os.mkdir(directory)  # 如果目录不存在，则创建它
    print(f"'{directory}' created" )
else:
    print(f" '{directory}' exists")

item_res_train, cate_res_train,  item_res_valid, cate_res_valid, item_res_test, cate_res_test,  usernum,  itemnum, catenum= datatimenormalizelast(User)

num_batch = len(item_res_train) // args.batch_size
cc = 0.0
for u in item_res_train:
    cc += len(item_res_train[u])
print('average item sequence length: %.2f' % (cc / len(item_res_train)))



item_res_all = dict()
cate_res_all = dict()

item_res_eval = dict()
cate_res_eval = dict()
for user in range(1,usernum+1):
    item_res_all[user] = item_res_train[user] + item_res_valid[user] + item_res_test[user]
    cate_res_all[user] = cate_res_train[user] + cate_res_valid[user] + cate_res_test[user]

for user in range(1,usernum+1):
    item_res_eval[user] = item_res_train[user] + item_res_valid[user] 
    cate_res_eval[user] = cate_res_train[user] + cate_res_valid[user] 



# time_point = generate_timepointfromtime(time_res_all, args.time_bucket)



et = datetime.datetime.now()
print("duration ", et-st)
dataloader = lastslow_newDataloader()
try:
    print('try')
    with open('./dataset/%s/dataloader_%d_train%d.pickle'%(args.datasets, args.maxlen,  args.batch_size),"rb") as f:
        wswe= pickle.load(f)
    dataloader.load(wswe, args)

except Exception as e:
    print(e.args)
    print('init1')
    dataloader.init(item_res_train, cate_res_train,  usernum, args.batch_size, 5, args.maxlen, args.incatemaxlen)

    with open('./dataset/%s/dataloader_%d_train%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"wb") as f:
        pickle.dump(dataloader.dataset, f)

[usetensors, itemytensors, cateytensors,  itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors, itemyseqtensors, cateyseqtensors] = zip(*dataloader.dataset)

traindataset = mydataset(usetensors, itemytensors, cateytensors, itemseqtensors, cateseqtensors,  padincateitemtensors,  padcatesettensors, itemyseqtensors, cateyseqtensors, args)

evaldataloader = lastslow_newevalDataloader()
try:
    print('try')
    with open('./dataset/%s/dataloader_%d_eval%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"rb") as f:
        wswe= pickle.load(f)
    evaldataloader.load(wswe, args)

except Exception as e:
    print(e.args)
    print('init2')
    evaldataloader.init(item_res_eval, cate_res_eval,  item_res_valid, usernum, args.batch_size, 5, args.maxlen, args.incatemaxlen)
    with open('./dataset/%s/dataloader_%d_eval%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"wb") as f:
            pickle.dump(evaldataloader.dataset, f)

[usetensors, itemytensors, cateytensors, itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors,itemyseqtensors, cateyseqtensors] = zip(*evaldataloader.dataset)

evaldataset = mydataset(usetensors, list(itemytensors), list(cateytensors), itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors, itemyseqtensors,  cateyseqtensors,  args)

testdataloader = lastslow_newevalDataloader()
try:
    print('try')
    with open('./dataset/%s/dataloader_%d_test%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"rb") as f:
        wswe= pickle.load(f)
    testdataloader.load(wswe, args)

except Exception as e:
    print(e.args)
    print('init2')
    testdataloader.init(item_res_all, cate_res_all, item_res_test, usernum, args.batch_size, 5, args.maxlen, args.incatemaxlen)
    with open('./dataset/%s/dataloader_%d_test%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"wb") as f:
            pickle.dump(testdataloader.dataset, f)







[usetensors, itemytensors, cateytensors, itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors,itemyseqtensors, cateyseqtensors] = zip(*testdataloader.dataset)

testdataset = mydataset(usetensors, list(itemytensors), list(cateytensors), itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors, itemyseqtensors,  cateyseqtensors,  args)


mydata = DataModule(traindataset, evaldataset, testdataset, args)



imap, imaps = catemap(args.dataset)
args.imap = imap




# trainer = pl.Trainer(devices=[args.cuda], accelerator="gpu", accumulate_grad_batches=1, max_epochs=args.num_epochs,  default_root_dir='./script/{}/{}//beamsize{}/lr{}'.format(args.model, args.datasets, args.beam_size,  args.lr))
# trainer.fit(recmodel, mydata)

Preparing data...
 './dataset/sub_taobao/' exists
average item sequence length: 30.57
duration  0:00:04.706557
try
try
try


In [12]:

import json
import numpy as np
import torch
import torch.nn as nn
import math

from torch.nn.init import xavier_uniform_, constant_, normal_

class PointWiseFeedForward(torch.nn.Module):
    def __init__(self, hidden_units, dropout_rate):

        super(PointWiseFeedForward, self).__init__()

        self.conv1 = torch.nn.Conv1d(hidden_units, hidden_units, kernel_size=1)
        self.dropout1 = torch.nn.Dropout(p=dropout_rate)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv1d(hidden_units, hidden_units, kernel_size=1)
        self.dropout2 = torch.nn.Dropout(p=dropout_rate)

    def forward(self, inputs):
        outputs = self.dropout2(self.conv2(self.relu(self.dropout1(self.conv1(inputs.transpose(-1, -2))))))
        outputs = outputs.transpose(-1, -2) # as Conv1D requires (N, C, Length)
        outputs += inputs
        return outputs


class cudaSASRec(torch.nn.Module):
    def __init__(self, user_num, item_num, args):
        super(cudaSASRec, self).__init__()

        self.user_num = user_num
        self.item_num = item_num
        self.dev = args.device
        self.sigmoid = torch.nn.Sigmoid()
        self.emb = args.hidden_units
        self.embedding_size = args.hidden_units
        # TODO: loss += args.l2_emb for regularizing embedding vectors during training
        # https://stackoverflow.com/questions/42704283/adding-l1-l2-regularization-in-pytorch
        self.item_emb = torch.nn.Embedding(self.item_num+1, args.hidden_units, padding_idx=0)
        self.pos_emb = torch.nn.Embedding(args.maxlen, args.hidden_units) # TO IMPROVE
        self.beha_emb = torch.nn.Embedding(5, args.hidden_units)
        self.user_emb = torch.nn.Embedding(self.user_num, args.hidden_units)

        # define the module feature gating need
        self.w1 = nn.Linear(args.hidden_units, args.hidden_units)
        self.w2 = nn.Linear(args.hidden_units, args.hidden_units)
        self.b = nn.Parameter(torch.zeros(args.hidden_units), requires_grad=True).to(self.dev)

        # define the module instance gating need
        self.w3 = nn.Linear(args.hidden_units, 1, bias=False)
        self.w4 = nn.Linear(args.hidden_units, args.maxlen, bias=False)
        self.w5 = nn.Linear(args.hidden_units, args.hidden_units)
        self.w6 = nn.Linear(args.hidden_units, args.hidden_units)

        self.emb_dropout = torch.nn.Dropout(p=args.dropout_rate)

        self.attention_layernorms = torch.nn.ModuleList() # to be Q for self-attention
        self.attention_layers = torch.nn.ModuleList()
        self.forward_layernorms = torch.nn.ModuleList()
        self.forward_layers = torch.nn.ModuleList()
        self.feed_forward = PointWiseFeedForward(args.hidden_units, args.dropout_rate)

        self.last_layernorm = torch.nn.LayerNorm(args.hidden_units, eps=1e-8)
        self.dropout = torch.nn.Dropout(p = args.dropout_rate)

        for _ in range(args.num_blocks):
            new_attn_layernorm = torch.nn.LayerNorm(args.hidden_units, eps=1e-8)
            self.attention_layernorms.append(new_attn_layernorm)

            new_attn_layer =  torch.nn.MultiheadAttention(args.hidden_units,
                                                            args.num_heads,
                                                            args.dropout_rate)
            self.attention_layers.append(new_attn_layer)

            new_fwd_layernorm = torch.nn.LayerNorm(args.hidden_units, eps=1e-8)
            self.forward_layernorms.append(new_fwd_layernorm)

            new_fwd_layer = PointWiseFeedForward(args.hidden_units, args.dropout_rate)
            self.forward_layers.append(new_fwd_layer)

            # self.pos_sigmoid = torch.nn.Sigmoid()
            # self.neg_sigmoid = torch.nn.Sigmoid()
        self.apply(self._init_weights)

    def log2feats(self, seqs, mask):
        # seqs = self.item_emb(log_seqs)

        # beha = self.beha_emb(beha_seqs)

        seqs  = seqs

        seqs *= self.item_emb.embedding_dim ** 0.5

        positions = np.tile(np.array(range(seqs.shape[1])), [seqs.shape[0], 1])
        seqs += self.pos_emb(torch.LongTensor(positions).to(self.dev))
        seqs = self.emb_dropout(seqs)

        timeline_mask = mask
        seqs *= ~timeline_mask.unsqueeze(-1) # broadcast in last dim

        tl = seqs.shape[1] # time dim len for enforce causality
        attention_mask = ~torch.tril(torch.ones((tl, tl), dtype=torch.bool, device=self.dev))

        for i in range(len(self.attention_layers)):
            seqs = torch.transpose(seqs, 0, 1)
            Q = self.attention_layernorms[i](seqs)
            mha_outputs, _ = self.attention_layers[i](Q, seqs, seqs, 
                                            attn_mask=attention_mask)
                                            # key_padding_mask=timeline_mask
                                            # need_weights=False) this arg do not work?
            seqs = Q + mha_outputs
            seqs = torch.transpose(seqs, 0, 1)

            seqs = self.forward_layernorms[i](seqs)
            seqs = self.forward_layers[i](seqs)
            seqs *=  ~timeline_mask.unsqueeze(-1)

        log_feats = self.last_layernorm(seqs) # (U, T, C) -> (U, -1, C)

        return log_feats

    def predict(self, user_ids, log_seqs, item_indices): # for inference
        log_feats = self.log2feats(log_seqs) # user_ids hasn't been used yet

        final_feat = log_feats[:, -1, :] # only use last QKV classifier, a waste

        item_embs = self.item_emb(item_indices) # (U, I, C)

        logits = torch.einsum('bcd,bd -> bc',item_embs, final_feat)
        # logits = item_embs.matmul(final_feat.unsqueeze(-1)).squeeze(-1)

        # preds = self.pos_sigmoid(logits) # rank same item list for different users

        return  logits # preds # (U, I)
    
    def catepredict(self, user_ids, log_seqs, all_indices): # for inference
        log_feats = self.log2feats(log_seqs) # user_ids hasn't been used yet

        final_feat = log_feats[:, -1, :] # only use last QKV classifier, a waste

        item_embs = self.item_emb(all_indices) # (U, I, C)

        logits = torch.einsum('cd,bd -> bc',item_embs, final_feat)
        # logits = item_embs.matmul(final_feat.unsqueeze(-1)).squeeze(-1)

        # preds = self.pos_sigmoid(logits) # rank same item list for different users

        return  logits, final_feat # preds # (U, I)
    
    def head(self, logits, indices):
        if indices is None:
            indices = torch.arange(0, self.item_num+1).to(self.dev)
            item_embs = self.item_emb(indices) # (U, I, C)
            logits = torch.einsum('cd,bd -> bc',item_embs, logits)
        else:
            item_embs = self.item_emb(indices) # (U, I, C)
            logits = torch.einsum('bcd,bd -> bc',item_embs, logits)
        return logits

    def trainq(self, log_seqs, seq_y, seq_neg):
        if seq_neg is None:
            seq_neg = torch.arange(0,self.item_num+1).to(self.dev)
            item_embs = self.item_emb(seq_neg)
            logits = torch.einsum('bnd, cd->bnc',log_seqs, item_embs)
        else:
            neg_embs = self.item_emb(seq_neg)
            seqy_embs = self.item_emb(seq_y)
            neg_logits = torch.einsum('bnd, cd -> bnc', log_seqs, neg_embs)
            pos_logits = torch.einsum('bnd, bnd->bn', log_seqs, seqy_embs)
            logits = torch.cat((pos_logits.unsqueeze(-1), neg_logits), -1)
        return logits


    def feature_gating(self, seq_item_embedding, user_embedding):
        """

        choose the features that will be sent to the next stage(more important feature, more focus)


        """

        batch_size, seq_len, embedding_size = seq_item_embedding.size()
        seq_item_embedding_value = seq_item_embedding

        seq_item_embedding = self.w1(seq_item_embedding)
        # batch_size * seq_len * embedding_size
        user_embedding = self.w2(user_embedding)
        # batch_size * embedding_size
        user_embedding = user_embedding.unsqueeze(1).repeat(1, seq_len, 1)
        # batch_size * seq_len * embedding_size

        user_item = self.sigmoid(seq_item_embedding + user_embedding + self.b)
        # batch_size * seq_len * embedding_size

        user_item = torch.mul(seq_item_embedding_value, user_item)
        # batch_size * seq_len * embedding_size

        return user_item



    def feature_attention(self, seq_item_embedding, user_embedding):
        """

        choose the features that will be sent to the next stage(more important feature, more focus)

        """

        batch_size, seq_len, embedding_size = seq_item_embedding.size()
        seq_item_embedding_value = seq_item_embedding

        seq_item_embedding = self.w1(seq_item_embedding)
        # batch_size * seq_len * embedding_size
        user_embedding = self.w2(user_embedding)
        # batch_size * embedding_size
        user_embedding = user_embedding.unsqueeze(1).repeat(1, seq_len, 1)
        # batch_size * seq_len * embedding_size

        user_item = self.sigmoid(seq_item_embedding + user_embedding + self.b)
        # batch_size * seq_len * embedding_size

        user_item = torch.mul(seq_item_embedding_value, user_item)
        # batch_size * seq_len * embedding_size

        return user_item
    
    def instance_attention(self,  user_item, user_embedding, mask, item_seq):
        """

        choose the last click items that will influence the prediction( more important more chance to get attention)
        """

        user_embedding_value = user_item

        user_item = self.w6(user_item)
        # batch_size * seq_len * embedding_size

        user_embedding = self.w5(user_embedding).unsquueze(1)
        # batch_size * embedding_size
        # print(user_item.shape, user_embedding.shape)
        score = torch.einsum('bld,bhd->blh',user_item, user_embedding.repeat(1, user_item.shape[1],1))

# 
        # mask = mask.repeat(self.head_num, 1 ).unsqueeze(1)
        # print(mask.shape)
        # if mask is not None:

        #     if score.dtype == torch.float16:
        #         score = score.masked_fill(mask == 0, -65500)
        #     else:
        #         score = score.masked_fill(mask == 0, -torch.inf)
        
        # if torch.rand(1)>0.99:    
        #     print('mask:',mask)
        #     print('score:',score)

        score += mask
        score = self.dropout(nn.functional.softmax(score, dim=-1))


        # instance_score = self.sigmoid(user_item + user_embedding).squeeze(-1)
        # batch_size * seq_len * 1

        output = torch.einsum('blh,bhd->bld',score, user_embedding_value + item_seq)
        # output = torch.mul(score.unsqueeze(2), user_embedding_value)
        # batch_size * seq_len * embedding_size


        # output = torch.div(output.sum(dim=1), instance_score.sum(dim=1).unsqueeze(1))
            # batch_size * embedding_size
        # else:
            # # for max_pooling
            # index = torch.max(instance_score, dim=1)[1]
            # # batch_size * 1
            # output = self.gather_indexes(output, index)
            # # batch_size * seq_len * embedding_size ==>> batch_size * embedding_size
        output = self.feed_forward(output)

        return output


    def instance_gating(self, user_item, user_embedding):
        """

        choose the last click items that will influence the prediction( more important more chance to get attention)
        """

        user_embedding_value = user_item

        user_item = self.w3(user_item)
        # batch_size * seq_len * 1

        user_embedding = self.w4(user_embedding).unsqueeze(2)
        
        # batch_size * seq_len * 1

        instance_score = self.sigmoid(user_item + user_embedding).squeeze(-1)
        # batch_size * seq_len * 1
        output = torch.mul(instance_score.unsqueeze(2), user_embedding_value)
        # batch_size * seq_len * embedding_size


        output = torch.div(output.sum(dim=1), instance_score.sum(dim=1).unsqueeze(1))
            # batch_size * embedding_size
        # else:
            # # for max_pooling
            # index = torch.max(instance_score, dim=1)[1]
            # # batch_size * 1
            # output = self.gather_indexes(output, index)
            # # batch_size * seq_len * embedding_size ==>> batch_size * embedding_size

        return output


    def heads(self, logits, indices):
        if indices is None:
            indices = torch.arange(0, self.item_num+1).to(self.dev)
            item_embs = self.item_emb(indices) # (U, I, C)
            logits = torch.einsum('cd,bhd -> bhc',item_embs, logits)
        else:
            item_embs = self.item_emb(indices) # (U, I, C)
            logits = torch.einsum('bcd,bhd -> bhc',item_embs, logits)
        return logits
    
    def _init_weights(self, module):
        if isinstance(module, nn.Embedding):
            normal_(module.weight.data, 0., 1 / self.embedding_size)
        elif isinstance(module, nn.Linear):
            xavier_uniform_(module.weight.data)
            if module.bias is not None:
                constant_(module.bias.data, 0)


import pytorch_lightning as pl
from collections import defaultdict
from new_metric import *

class starcateseq(torch.nn.Module):
    def __init__(self, embedding_dim, cate_embedding_dim,  dropout_rate,  args):
        super(starcateseq, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout_rate)
        self.lastk =args.lastk
        self.norm = torch.nn.LayerNorm(args.hidden_size)
        self.embedding_dim = embedding_dim

        self.cate_embedding_dim = cate_embedding_dim
        # self.Q_w = torch.nn.Linear(embedding_dim, hidden_size)
        # self.K_w = torch.nn.Linear(embedding_dim, hidden_size)
        # self.V_w = torch.nn.Linear(embedding_dim, hidden_size)
        self.cate_querynetwork = nn.Linear(embedding_dim, args.hidden_size)
        self.cate_keynetwork = nn.Linear(embedding_dim, args.hidden_size)
        self.item_valuenetwork = nn.Linear(embedding_dim, embedding_dim)
        # #不需要归零化
        # self.weights = nn.Parameter(torch.randn( self.lastk, hidden_size), requires_grad=True)
        
        self.feed_forward = PointWiseFeedForward(args.hidden_size, dropout_rate)

        self.hidden_size = embedding_dim

        self.head_num = 1


        assert self.hidden_size%self.head_num ==0
        self.head_size = args.hidden_size // self.head_num

        self.apply(self._init_weights)

        # self.cate_querynetwork.weight.data.normal_(mean=0.0, std=0.02)
        # self.item_keynetwork.weight.data.normal_(mean=0.0, std=0.02)
        # self.item_valuenetwork.weight.data.normal_(mean=0.0, std=0.02)
        # self.weights.data.normal_(mean=0.0, std=0.02)



    def _init_weights(self, module):
        if isinstance(module, nn.Embedding):
            normal_(module.weight.data, 0., 1 / self.embedding_size)
        elif isinstance(module, nn.Linear):
            xavier_uniform_(module.weight.data)
            if module.bias is not None:
                constant_(module.bias.data, 0)

    def  forward(self, item_seq, cate_seq, cate_y, mask):
        cate_query = self.cate_querynetwork(cate_y)
        
        cate_key = self.cate_keynetwork(cate_seq)

        item_value = self.item_valuenetwork(item_seq)



        cate_query_ = torch.cat(torch.split(cate_query, self.head_size, dim=2), dim=0)
        cate_key_ = torch.cat(torch.split(cate_key, self.head_size, dim=2), dim=0)
        item_value_ = torch.cat(torch.split(item_value, self.head_size, dim=2), dim=0)

        # test_item_emb = self.item_embedding.weight

        att_0 = torch.einsum('bnd, bhd -> bnh', cate_query_, cate_key_)

        scores = att_0 \
                / math.sqrt(self.cate_embedding_dim) 

        # 2. dealing with padding and softmax.
        # get padding masks
        # mask = (x > 0)

        scores =+ mask

        # mask = mask.repeat(self.head_num, 1 ).unsqueeze(1)
        # # print(mask.shape)
        # if mask is not None:
        #     # assert len(mask.shape) == 2
        #     #mask [a,b] ->[a,1,b,b]
        #     # mask = (mask[:,:,None] & mask[:,None,:]).unsqueeze(1)
        #     if scores.dtype == torch.float16:
        #         scores = scores.masked_fill(mask == 0, -65500)
        #     else:
        #         scores = scores.masked_fill(mask == 0, -torch.inf)

        c_attn = self.dropout(nn.functional.softmax(scores, dim=-1))

        # item_key = self.item_keynetwork(item_seq[:, -self.lastk: ,:])
        # item_value = self.item_valuenetwork(item_seq[:, -self.lastk: ,:])
        # item_key = self.item_keynetwork(item_seq)
        

        att_out = torch.einsum('bnh, bhd -> bnd', c_attn, item_value_)
        # att_1 = torch.einsum('bn, nd -> bd', att_0, self.weights)

        att_out = torch.cat(torch.split(att_out, item_seq.shape[0], dim=0), dim=2) # div batch_size

        att_value = self.feed_forward(att_out)
        # att_value =self.feed_forward(att_out.unsqueeze(1)).squeeze()


        # (num_head * N, T, C / num_head) -> (N, T, C)
        

        return att_value


In [13]:

class casr(pl.LightningModule):
    def __init__(self, itemmodel, catemodel,  args
        ):
        super().__init__()
        self.teachingflag = True
        self.m_logsoftmax = torch.nn.LogSoftmax(dim=1)
        self.loss = torch.nn.CrossEntropyLoss()
        self.args = args
        self.itemmodel = itemmodel
        self.catemodel = catemodel
        self.starmodel = starcateseq(args.hidden_units, args.hidden_units, args.dropout_rate, args)
         

        self.train_epoch = 0
        self.dropout = torch.nn.Dropout(p=args.dropout_rate)

    def get_attention_mask( item_seq):
        """Generate left-to-right uni-directional attention mask for multi-head attention."""
        attention_mask = (item_seq > 0).long()
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # torch.int64
        # mask for left-to-right unidirectional
        max_len = attention_mask.size(-1)
        attn_shape = (1, max_len, max_len)
        subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1)  # torch.uint8
        subsequent_mask = (subsequent_mask == 0).unsqueeze(1)
        subsequent_mask = subsequent_mask.long().to(item_seq.device)

        extended_attention_mask = extended_attention_mask * subsequent_mask
        # extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        return extended_attention_mask
    
    def training_step(self, batch, batch_idx):
        args = self.args
        self.train_epoch +=1
        if self.train_epoch>8:
            self.teachingflag = False
        user_seq_batch, item_y_batch, cate_y_batch,  item_seq_batch, cate_seq_batch,  pad_incateitem_seq_batch, pad_cateset_seq_batch, itemy_seq_batch, catey_seq_batch = batch


        item_seq_batch = item_seq_batch.squeeze(0)
        item_y_batch = item_y_batch.squeeze(0)
        cate_seq_batch = cate_seq_batch.squeeze(0)
        cate_y_batch = cate_y_batch.squeeze(0)
        catey_seq_batch = catey_seq_batch.squeeze(0)
        itemy_seq_batch = itemy_seq_batch.squeeze(0)

        valid_numpy = np.array(itemy_seq_batch.to('cpu'))>0
        label_numpy = valid_numpy.nonzero()

        user_batch = torch.from_numpy(np.array(user_seq_batch)).unsqueeze(-1.1).to(self.args.device)

        cate_seq = self.catemodel.item_emb(cate_seq_batch)
        
        mask = (item_seq_batch>0)
        starmask = self.get_attention_mask(item_seq_batch).squeeze()

        cate_logits = self.catemodel.log2feats(cate_seq, mask)

        cateuserinput = self.catemodel.feature_gating(self.catemodel.item_emb(cate_seq_batch), self.catemodel.user_emb(user_batch))
        cateuserinput = self.catemodel.instance_attention(cateuserinput, self.catemodel.user_emb(user_batch), starmask, self.catemodel.item_emb(cate_seq_batch))

        cate_input = cate_logits + cateuserinput
        

        cate_outputs = self.itemmodel.trainq(cate_input, itemy_seq_batch, None)
        
        valid_logits = outputs[label_numpy]
        valid_labels = catey_seq_batch.squeeze()[label_numpy]
        cateloss = self.loss(cate_outputs, cate_y_batch)

        

        #behasum1
        # cate_input = self.starmodel(self.itemmodel.item_emb(item_seq_batch), cate_seq, self.catemodel.item_emb(cate_y_batch.reshape(-1,1)), mask)
        cate_input = self.starmodel(self.itemmodel.item_emb(item_seq_batch), cate_seq, self.catemodel.item_emb(catey_seq_batch), starmask)    

        starinput = cate_input.squeeze()

        userinput = self.itemmodel.feature_gating(self.itemmodel.item_emb(item_seq_batch), self.itemmodel.user_emb(user_batch))
        userinput = self.itemmodel.instance_attention(userinput, self.itemmodel.user_emb(user_batch), starmask, self.itemmodel.item_emb(item_seq_batch))


        output = starinput + userinput 

        # test_item_emb = self.itemmodel.item_emb.weight

        # logits = torch.matmul(output, test_item_emb.transpose(0, 1))

        # itemloss = self.loss(logits, item_y_batch)

        outputs = self.itemmodel.trainq(output, itemy_seq_batch, None)

       


        valid_logits = outputs[label_numpy]
        valid_labels = itemy_seq_batch.squeeze()[label_numpy]

        itemloss = self.loss(valid_logits, valid_labels)

        # for param in self.itemmodel.item_emb.parameters(): itemloss += args.l2_emb * torch.norm(param)


        loss = 0.8*itemloss + 0.2*cateloss

        # prelogit_1 = self.itemmodel.head(item_logits[:,-1,:], None)
        # prelogit_2 = self.itemmodel.head(starinput, None)
        # predicts = prelogit_1 + prelogit_2
        # recall_5, mrr_5 = evaluate(predicts, item_y_batch, torch.tensor([True]).expand_as(item_y_batch).to(self.args.device), k=args.recall)
        # caterecall_5, catemrr_5 = evaluate(cate_outputs, cate_y_batch, torch.tensor([True]).expand_as(valid_labels).to(self.args.device), k=args.recall)


        return {'loss':loss}


    def training_epoch_end(self, training_step_outputs):
        loss = torch.stack([o['loss'] for o in training_step_outputs], 0).mean()
        self.log('train_loss', loss)


    def validation_step(self, batch, batch_idx):

        user_seq_batch, item_y_batch, cate_y_batch,  item_seq_batch, cate_seq_batch,  pad_incateitem_seq_batch, pad_cateset_seq_batch, itemy_seq_batch, catey_seq_batch = batch


        item_seq_batch = item_seq_batch.squeeze(0)
        item_y_batch = item_y_batch.squeeze(0)
        cate_seq_batch = cate_seq_batch.squeeze(0)
        cate_y_batch = cate_y_batch.squeeze(0)
        catey_seq_batch = catey_seq_batch.squeeze(0)
        itemy_seq_batch = itemy_seq_batch.squeeze(0)

        user_batch = torch.from_numpy(np.array(user_seq_batch)).unsqueeze(-1.1).to(self.args.device)

        cate_seq = self.catemodel.item_emb(cate_seq_batch)

        mask = (item_seq_batch>0)

        starmask = self.get_attention_mask(item_seq_batch).squeeze()

        cate_logits = self.catemodel.log2feats(cate_seq, mask)

        cateuserinput = self.catemodel.feature_gating(self.catemodel.item_emb(cate_seq_batch), self.catemodel.user_emb(user_batch))
        cateuserinput = self.catemodel.instance_attention(cateuserinput, self.catemodel.user_emb(user_batch), starmask, self.catemodel.item_emb(cate_seq_batch))

        cate_input = cate_logits[:, -1, :] + cateuserinput[:,-1,:]
        
        # if batch_idx %100==0:
        #     print('cate: user_input:{}, cate_logits:{}'.format(userinput, cate_logits[torch.arange(cate_logits.size(0)), item_len_batch, :]))
        
        cate_outputs = self.catemodel.head(cate_input, None).squeeze()



        pred_item_prob1 = None
        

        log_prob_cate_short = self.m_logsoftmax(cate_outputs)
        log_prob_cate_short, pred_cate_index = torch.topk(log_prob_cate_short, args.beam_size, dim=-1)
        
        pred_cate_index = pred_cate_index.detach()
        log_prob_cate_short = log_prob_cate_short

        prob_cate_short = self.m_logsoftmax(cate_outputs)
        prob_cate_short, pred_cate_index = torch.topk(prob_cate_short, args.beam_size, dim=-1)
        
        pred_cate_index = pred_cate_index.detach()
        # prob_cate_short = log_prob_cate_short

        pred_cate_emb = self.catemodel.item_emb(pred_cate_index)
        prob_cate_beam = prob_cate_short[:, :args.beam_size]

        # prob_emb = pred_cate_emb * prob_cate_beam
        # prob_emb = torch.einsum('bnd, bn->bnd', pred_cate_emb, prob_cate_beam)

        cate_input = self.starmodel(self.itemmodel.item_emb(item_seq_batch), cate_seq , pred_cate_emb, starmask[:,-1,:].unsqueeze(1))
    

        # starinput = torch.einsum('bnd, bn->bd', cate_input, prob_cate_beam)
    
        userinput = self.itemmodel.feature_gating(self.itemmodel.item_emb(item_seq_batch), self.itemmodel.user_emb(user_batch))

        userinput = self.itemmodel.instance_attention(userinput, self.itemmodel.user_emb(user_batch), starmask, self.itemmodel.item_emb(item_seq_batch))


        starinput = cate_input
        # print(starinput.shape, userinput.shape )
        output = starinput + userinput[:,-1,:].unsqueeze(1) 

        # if batch_idx %100==0:
        #     print('item: user_input:{}, star_input:{}'.format(userinput, starinput))

        logit_2 = self.itemmodel.heads(output, None)

        logit_2_1 = self.m_logsoftmax2(logit_2)

        outputs = logit_2_1 + prob_cate_beam[:,:,None]

        pred_item_prob1 = outputs

        pred_item_prob1 = torch.logsumexp(pred_item_prob1, dim = 1)


    
        valid_labels = torch.ones((pred_item_prob1.shape[0])).long().to(args.device)*100


        metrics = evaluate(pred_item_prob1, item_y_batch, torch.tensor([True]).expand_as(valid_labels).to(self.args.device), recalls=args.recall)


        cate_recall_5, cate_mrr, cate_ndcg = evaluate_part(cate_outputs, cate_y_batch, torch.tensor([True]).expand_as(valid_labels).to(self.args.device), k=5)




        return {'metrics':metrics,  'validcate_recall_5':cate_recall_5}

    


    def validation_epoch_end(self, validation_step_outputs):
        keys = validation_step_outputs[0]['metrics'].keys()
        metric =  defaultdict(list)
        for o in validation_step_outputs:
            for k in keys:
                metric[k].append(o['metrics'][k])

        for k in keys:
            self.log(f'Val:{k}', torch.Tensor(metric[k]).mean(), on_epoch=True)

        validcate_recall_5 = torch.stack([o['validcate_recall_5'] for o in validation_step_outputs], 0).mean()
        self.log('validcate_recall_5', validcate_recall_5, on_epoch=True)

        

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.args.lr, betas=(0.9, 0.98))
        return optimizer




In [14]:
itemmodel = cudaSASRec(usernum, itemnum, args)

catemodel = cudaSASRec(usernum, catenum, args )

recmodel = casr(itemmodel, catemodel, args)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

early_stop_callback = EarlyStopping(monitor="Val:Recall@20", min_delta=0.00, patience=20, verbose=False, mode="max")



# trainer.fit(recmodel, )

In [15]:
trainer = pl.Trainer(devices=[0], callbacks=[early_stop_callback], accelerator="gpu", accumulate_grad_batches=1, max_epochs=200,  default_root_dir='./script_test/{}/casr_batch_beha_user3/both/beam{}/recall{}/lr{}'.format(args.dataset ,args.beam_size, args.recall, args.lr))
trainer.fit(recmodel, mydata)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9,10,11]

  | Name         | Type             | Params
--------------------------------------------------
0 | m_logsoftmax | LogSoftmax       | 0     
1 | loss         | CrossEntropyLoss | 0     
2 | itemmodel    | cudaSASRec       | 6.0 M 
3 | catemodel    | cudaSASRec       | 2.2 M 
4 | starmodel    | starcateseq      | 20.9 K
5 | dropout      | Dropout          | 0     
--------------------------------------------------
8.2 M     Trainable params
0         Non-trainable params
8.2 M     Total params
32.716    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/dev-data/zzz/anaconda3/envs/mulbehavior/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [14]:
for i in traindataset:
    break

In [19]:
len(i[0]), len(i[1])

(128, 128)

In [21]:
i[3].shape

torch.Size([128, 50])

In [23]:
def get_attention_mask( item_seq):
    """Generate left-to-right uni-directional attention mask for multi-head attention."""
    attention_mask = (item_seq > 0).long()
    extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # torch.int64
    # mask for left-to-right unidirectional
    max_len = attention_mask.size(-1)
    attn_shape = (1, max_len, max_len)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1)  # torch.uint8
    subsequent_mask = (subsequent_mask == 0).unsqueeze(1)
    subsequent_mask = subsequent_mask.long().to(item_seq.device)

    extended_attention_mask = extended_attention_mask * subsequent_mask
    # extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
    extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
    return extended_attention_mask

In [47]:
a = get_attention_mask(i[3])
a.shape

torch.Size([128, 1, 50, 50])

In [30]:
i[3][0]

tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 60201, 11064, 60201,  7847, 17790, 52249, 58754,
        13760, 33071,  9790, 50774,  5915, 41111,  5409, 33071,  6571, 37840])

In [33]:
a[0][0][-5]

tensor([-10000., -10000., -10000., -10000., -10000., -10000., -10000., -10000.,
        -10000., -10000., -10000., -10000., -10000., -10000., -10000., -10000.,
        -10000., -10000., -10000., -10000., -10000., -10000., -10000., -10000.,
        -10000., -10000., -10000., -10000., -10000., -10000., -10000., -10000.,
        -10000.,     -0.,     -0.,     -0.,     -0.,     -0.,     -0.,     -0.,
            -0.,     -0.,     -0.,     -0.,     -0.,     -0., -10000., -10000.,
        -10000., -10000.])

In [38]:
i[3].shape

torch.Size([128, 50])

In [51]:
mask = (i[3]>0)
scores = torch.ones((128,50,50))
t = a.squeeze()+scores

In [37]:
mask.unsqueeze(1).shape

torch.Size([128, 1, 1, 50])

In [42]:
# mask = mask.repeat(self.head_num, 1).unsqueeze(1)
# print(mask.shape)
mask = mask.unsqueeze(1)

if mask is not None:
    # assert len(mask.shape) == 2
    #mask [a,b] ->[a,1,b,b]
    # mask = (mask[:,:,None] & mask[:,None,:]).unsqueeze(1)
    if scores.dtype == torch.float16:
        scores = scores.masked_fill(mask == 0, -65500)
    else:
        scores = scores.masked_fill(mask == 0, -torch.inf)
scores

tensor([[[-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         ...,
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.]],

        [[-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         ...,
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.]],

        [[-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         ...,
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.]],

        ...,

        [[-inf, -inf, -inf,  ..., 1., 1., 1.],
         [-inf, -inf, -inf,  ..., 1., 1., 1.]

In [50]:
t.shape

torch.Size([128, 128, 50, 50])

In [52]:
scores[0][-5], t[0][-5]

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([-9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
         -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
         -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
         -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
         -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
         -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
         -9.9990e+03, -9.9990e+03, -9.9990e+03,  1.0000e+00,  1.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
          1.0000e+00, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03]))

In [53]:
ts = t.repeat(2, 1,1 )

ts[0][-5]

tensor([-9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03])

In [55]:
t[1][-3]

tensor([-9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03, -9.9990e+03,
        -9.9990e+03, -9.9990e+03, -9.9990e+03,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00, -9.9990e+03, -9.9990e+03])

In [1]:
import torch

# 定义大小为[B, N]的tensor
B = 5
N = 10
tensor = torch.randn(B, N)
tensor


tensor([[-0.6254,  0.4871,  0.2400,  2.1101, -0.0226,  0.8141, -0.0598,  2.1379,
         -1.6378, -0.2752],
        [-0.8653,  0.5151, -1.5794, -0.0190,  0.3191,  1.0822,  0.6756,  0.5978,
          0.1186, -1.5415],
        [-0.3698, -0.5624, -0.8845, -0.2472,  0.7751,  0.3503,  0.0771, -0.5835,
          1.3677,  0.1429],
        [-1.3079, -0.1241, -1.0141, -0.4082, -0.0472,  0.6189, -0.1567, -0.1290,
         -1.3208,  0.6384],
        [ 1.3198,  0.8771, -0.8397,  0.0062, -0.3539, -0.5559, -1.4857,  1.7359,
         -0.5486, -1.6193]])

In [2]:
# 生成大小为B的随机排列
permuted_indices = torch.randperm(B)

# 使用随机排列来重新排列tensor的B维度
shuffled_tensor = tensor[permuted_indices]

print(shuffled_tensor)

tensor([[-0.8653,  0.5151, -1.5794, -0.0190,  0.3191,  1.0822,  0.6756,  0.5978,
          0.1186, -1.5415],
        [-1.3079, -0.1241, -1.0141, -0.4082, -0.0472,  0.6189, -0.1567, -0.1290,
         -1.3208,  0.6384],
        [ 1.3198,  0.8771, -0.8397,  0.0062, -0.3539, -0.5559, -1.4857,  1.7359,
         -0.5486, -1.6193],
        [-0.6254,  0.4871,  0.2400,  2.1101, -0.0226,  0.8141, -0.0598,  2.1379,
         -1.6378, -0.2752],
        [-0.3698, -0.5624, -0.8845, -0.2472,  0.7751,  0.3503,  0.0771, -0.5835,
          1.3677,  0.1429]])


In [4]:
tensor, tensor.

tensor([[-0.6254,  0.4871,  0.2400,  2.1101, -0.0226,  0.8141, -0.0598,  2.1379,
         -1.6378, -0.2752],
        [-0.8653,  0.5151, -1.5794, -0.0190,  0.3191,  1.0822,  0.6756,  0.5978,
          0.1186, -1.5415],
        [-0.3698, -0.5624, -0.8845, -0.2472,  0.7751,  0.3503,  0.0771, -0.5835,
          1.3677,  0.1429],
        [-1.3079, -0.1241, -1.0141, -0.4082, -0.0472,  0.6189, -0.1567, -0.1290,
         -1.3208,  0.6384],
        [ 1.3198,  0.8771, -0.8397,  0.0062, -0.3539, -0.5559, -1.4857,  1.7359,
         -0.5486, -1.6193]])

In [7]:
labels = torch.arange(2 * 10, dtype=torch.long)
labels

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])